In [218]:
import pandas as pd
from datetime import datetime
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import Lasso
from sklearn.feature_selection import SelectKBest, f_classif
from scipy.stats import ttest_ind
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score

## 데이터 수집 및 전처리
### kosdaq

In [219]:
## kosdqp
## csv 파일로부터 재무제표 데이터를 불러옴
kosdaq1 = pd.read_csv("./코스닥데이터(연결재무제표).csv", encoding = "euc-kr")
kosdaq2 = pd.read_csv("./코스닥데이터(개별재무제표).csv", encoding = "euc-kr")
kosdaq1.columns = ["회사명", "거래소코드", "회계년도", "매출액증가율", "총자본증가율", "자기자본증가율", "순이익증가율", "유형자산증가율", "비유동자산증가율", "유동자산증가율", "재고자산증가율", "유동비율", "당좌비율", "현금비율", "순운전자본비율", "부채비율", "자기자본배율", "비유동비율", "유동자산구성비율", "자기자본구성비율", "매출액총이익률", "매출액순이익률", "총자본사업이익률", "총자본순이익률", "자기자본순이익률", "1주당순이익(원)", "1주당매출액(원)", "총자본회전률", "자기자본회전률", "비유동자산회전률", "재고자산회전률", "매출채권회전률", "매입채무회전률", "매입채무회전기간", "재고자산회전기간", "매출채권회전기간", "1회전기간", "운전가본회전률", "순운전자본회전률", "유형자산회전율", "유동자산회전률", "당좌자산회전률", "부가가치(백만원)", "총자본투자효율", "설비투자효율", "기계투자효율", "부가가치율", "노동소득분배율", "자본분배율", "이윤분배율", "상장폐지일", "소속코드", "상장일"]
kosdaq2.columns = ["회사명", "거래소코드", "회계년도", "매출액증가율", "총자본증가율", "자기자본증가율", "순이익증가율", "유형자산증가율", "비유동자산증가율", "유동자산증가율", "재고자산증가율", "유동비율", "당좌비율", "현금비율", "순운전자본비율", "부채비율", "자기자본배율", "비유동비율", "유동자산구성비율", "자기자본구성비율", "매출액총이익률", "매출액순이익률", "총자본사업이익률", "총자본순이익률", "자기자본순이익률", "1주당순이익(원)", "1주당매출액(원)", "총자본회전률", "자기자본회전률", "비유동자산회전률", "재고자산회전률", "매출채권회전률", "매입채무회전률", "매입채무회전기간", "재고자산회전기간", "매출채권회전기간", "1회전기간", "운전가본회전률", "순운전자본회전률", "유형자산회전율", "유동자산회전률", "당좌자산회전률", "부가가치(백만원)", "총자본투자효율", "설비투자효율", "기계투자효율", "부가가치율", "노동소득분배율", "자본분배율", "이윤분배율", "상장폐지일", "소속코드", "상장일"]

In [220]:
## 결측치를 개별재무제표로 대체함
kosdaq = kosdaq1.fillna(kosdaq2)

In [221]:
## 해당년도 재무제표가 존재하지 않는 기업 해당년도 삭제
# kosdaq.isna().sum()
kosdaq.dropna(subset=["매출액증가율"], inplace = True)
# kosdaq.isna().sum()

In [222]:
# 상장일 이전에 회계년도 데이터 삭제
kosdaq['상장일'] = pd.to_datetime(kosdaq['상장일'])
kosdaq['회계년도'] = pd.to_datetime(kosdaq['회계년도'], format='%Y/%m')
kosdaq = kosdaq.loc[kosdaq['회계년도'] >= kosdaq['상장일']]

In [223]:
## kosdaq 외국계 기업 종목코드 제거
kosdaq = kosdaq[kosdaq['거래소코드'] < 900000]

In [224]:
# 거래소코드 2개인 기업들 셀렉
grouped = kosdaq.groupby('회사명')['거래소코드'].nunique()
result = grouped[grouped > 1].index.tolist()
print(result)

['(주)알로이스', '(주)에스에이티', '(주)엔케이맥스', '(주)원바이오젠', '(주)원익피앤이', '(주)자비스', '(주)티에스트릴리온', '(주)포인트엔지니어링', '(주)휴럼', '에스케이오션플랜트(주)', '엠에프엠코리아(주)', '원텍(주)', '줌인터넷(주)']


In [225]:
## 거래소코드 2개인 기업 확인하여 삭제
codes = [271400, 60540, 262760, 278380, 131390, 230400, 284610, 176560, 284420, 251960, 216280,25440, 229480]
kosdaq = kosdaq.loc[~(kosdaq['거래소코드'].isin(codes))]

In [226]:
## 증가율 0 인 데이터 평균값으로 대체
kosdaq['자기자본증가율'] = kosdaq['자기자본증가율'].replace(0, np.nan)
mean_growth_rate = kosdaq['자기자본증가율'].mean()
kosdaq['자기자본증가율'] = kosdaq['자기자본증가율'].fillna(mean_growth_rate)
kosdaq['순이익증가율'] = kosdaq['순이익증가율'].replace(0, np.nan)
mean_growth_rate = kosdaq['순이익증가율'].mean()
kosdaq['순이익증가율'] = kosdaq['순이익증가율'].fillna(mean_growth_rate)
kosdaq['매출액증가율'] = kosdaq['매출액증가율'].replace(0, np.nan)
mean_growth_rate = kosdaq['매출액증가율'].mean()
kosdaq['매출액증가율'] = kosdaq['매출액증가율'].fillna(mean_growth_rate)
kosdaq['총자본증가율'] = kosdaq['총자본증가율'].replace(0, np.nan)
mean_growth_rate = kosdaq['총자본증가율'].mean()
kosdaq['총자본증가율'] = kosdaq['총자본증가율'].fillna(mean_growth_rate)
kosdaq['유형자산증가율'] = kosdaq['유형자산증가율'].replace(0, np.nan)
mean_growth_rate = kosdaq['유형자산증가율'].mean()
kosdaq['유형자산증가율'] = kosdaq['유형자산증가율'].fillna(mean_growth_rate)
kosdaq['비유동자산증가율'] = kosdaq['비유동자산증가율'].replace(0, np.nan)
mean_growth_rate = kosdaq['비유동자산증가율'].mean()
kosdaq['비유동자산증가율'] = kosdaq['비유동자산증가율'].fillna(mean_growth_rate)
kosdaq['유동자산증가율'] = kosdaq['유동자산증가율'].replace(0, np.nan)
mean_growth_rate = kosdaq['유동자산증가율'].mean()
kosdaq['유동자산증가율'] = kosdaq['유동자산증가율'].fillna(mean_growth_rate)
kosdaq['재고자산증가율'] = kosdaq['재고자산증가율'].replace(0, np.nan)
mean_growth_rate = kosdaq['재고자산증가율'].mean()
kosdaq['재고자산증가율'] = kosdaq['재고자산증가율'].fillna(mean_growth_rate)

In [227]:
# 거래소코드와 소속코드 문자열로 변환
kosdaq = kosdaq.astype({"거래소코드" : "str"})
kosdaq = kosdaq.astype({"소속코드" : "str"})

In [228]:
# 이상치 확인
kosdaq['매출액증가율']

q5 = np.percentile(kosdaq['매출액증가율'], 5)
q95 = np.percentile(kosdaq['매출액증가율'], 95)

outliers = kosdaq['매출액증가율'][(kosdaq['매출액증가율'] < q5) | (kosdaq['매출액증가율'] > q95)]

print("하위 5%:", q5)
print("상위 5%:", q95)
print("이상치 :", outliers)

하위 5%: -40.28
상위 5%: 78.68599999999996
이상치 : 7        84.57
26      -50.47
38      301.12
46      -45.85
47      226.35
         ...  
7636     91.21
7638    -57.02
7639    -42.66
7642     80.87
7646    603.96
Name: 매출액증가율, Length: 669, dtype: float64


In [229]:
# 윈저라이징 함수 만들기
def windsorize(data, lower_percentile=0.05, upper_percentile=0.95):
    if isinstance(data, pd.Series):
        data = pd.DataFrame(data)
    numeric_df = data.select_dtypes(include=np.number)
    lower_bound = np.percentile(numeric_df, lower_percentile * 100)
    upper_bound = np.percentile(numeric_df, upper_percentile * 100)
    windsorized_data = np.where(numeric_df < lower_bound, lower_bound, numeric_df)
    windsorized_data = np.where(windsorized_data > upper_bound, upper_bound, windsorized_data)
    return windsorized_data

In [230]:
# 윈저라이징 실행(이상치 제거)
for i in range(3, 50, 1):
    kosdaq[kosdaq.columns[i]] = windsorize(kosdaq[kosdaq.columns[i]])

In [231]:
# 윈저라이징 후 이상치 확인
kosdaq['매출액증가율']

q5 = np.percentile(kosdaq['매출액증가율'], 5)
q95 = np.percentile(kosdaq['매출액증가율'], 95)

outliers = kosdaq['매출액증가율'][(kosdaq['매출액증가율'] < q5) | (kosdaq['매출액증가율'] > q95)]

print("하위 5%:", q5)
print("상위 5%:", q95)
print("이상치:", outliers)

하위 5%: -40.28
상위 5%: 78.65719999999999
이상치: 7       78.686
38      78.686
47      78.686
50      78.686
78      78.686
         ...  
7548    78.686
7587    78.686
7636    78.686
7642    78.686
7646    78.686
Name: 매출액증가율, Length: 335, dtype: float64


In [232]:
# standard 스케일링
numeric_cols = kosdaq.select_dtypes(include=np.number).columns
scaler = StandardScaler()
kosdaq[numeric_cols] = scaler.fit_transform(kosdaq[numeric_cols])

print(kosdaq.head())

        회사명  거래소코드       회계년도    매출액증가율    총자본증가율   자기자본증가율    순이익증가율   
0  (주)CMG제약  58820 2018-12-01  0.267103 -0.329883 -0.251115  1.124574  \
1  (주)CMG제약  58820 2019-12-01  0.182738  2.461924  2.084707 -1.427800   
2  (주)CMG제약  58820 2020-12-01 -0.074763 -0.451324 -0.446753 -1.983355   
3  (주)CMG제약  58820 2021-12-01 -0.148625 -0.436856 -0.396974  2.546591   
4  (주)CMG제약  58820 2022-12-01  0.208149 -0.212389 -0.460678  0.294699   

    유형자산증가율  비유동자산증가율   유동자산증가율  ...   총자본투자효율    설비투자효율    기계투자효율     부가가치율   
0 -0.273520  0.036829 -0.406337  ... -0.046738 -0.176940 -0.346626  0.465836  \
1  0.606500  0.344726  2.120575  ... -0.394861 -0.148162 -0.352061  0.421143   
2  0.169694  0.101700 -0.502793  ... -0.480630 -0.170151 -0.347129  0.215627   
3 -0.382989  1.074120 -0.829672  ... -0.409996 -0.105667 -0.330288  0.262012   
4  0.155810 -0.291319 -0.184250  ... -0.570003 -0.174424 -0.338809 -0.022055   

    노동소득분배율     자본분배율     이윤분배율  상장폐지일  소속코드        상장일  
0  0.052136  0.247429 

In [233]:
# 각 컬럼별 평균값
kosdaq[numeric_cols].mean()

매출액증가율       1.697579e-17
총자본증가율       1.697579e-17
자기자본증가율     -1.909776e-17
순이익증가율       1.114036e-17
유형자산증가율     -2.228072e-17
비유동자산증가율     7.639104e-17
유동자산증가율      6.365920e-18
재고자산증가율     -1.644529e-17
유동비율        -1.273184e-16
당좌비율        -1.697579e-16
현금비율         1.188305e-16
순운전자본비율     -6.790314e-17
부채비율         1.612700e-16
자기자본배율       5.517130e-17
비유동비율        3.098081e-16
유동자산구성비율    -2.376610e-16
자기자본구성비율     4.519803e-16
매출액총이익률     -5.517130e-17
매출액순이익률      5.941525e-17
총자본사업이익률    -1.273184e-17
총자본순이익률      8.487893e-18
자기자본순이익률     1.060987e-17
1주당순이익(원)    1.273184e-17
1주당매출액(원)   -8.912288e-17
총자본회전률       2.291731e-16
자기자본회전률      8.063498e-17
비유동자산회전률     1.007937e-16
재고자산회전률     -2.758565e-17
매출채권회전률     -4.243946e-17
매입채무회전률      2.121973e-18
매입채무회전기간    -1.156475e-16
재고자산회전기간    -7.267758e-17
매출채권회전기간     2.546368e-17
1회전기간       -1.856727e-17
운전가본회전률     -2.970763e-17
순운전자본회전률    -5.092736e-17
유형자산회전율      8.063498e-17
유동자산회전률      8.487893e-18
당좌자산회전률     

In [234]:
# 각 컬럼별 표준편차
kosdaq[numeric_cols].std()

매출액증가율       1.000075
총자본증가율       1.000075
자기자본증가율      1.000075
순이익증가율       1.000075
유형자산증가율      1.000075
비유동자산증가율     1.000075
유동자산증가율      1.000075
재고자산증가율      1.000075
유동비율         1.000075
당좌비율         1.000075
현금비율         1.000075
순운전자본비율      1.000075
부채비율         1.000075
자기자본배율       1.000075
비유동비율        1.000075
유동자산구성비율     1.000075
자기자본구성비율     1.000075
매출액총이익률      1.000075
매출액순이익률      1.000075
총자본사업이익률     1.000075
총자본순이익률      1.000075
자기자본순이익률     1.000075
1주당순이익(원)    1.000075
1주당매출액(원)    1.000075
총자본회전률       1.000075
자기자본회전률      1.000075
비유동자산회전률     1.000075
재고자산회전률      1.000075
매출채권회전률      1.000075
매입채무회전률      1.000075
매입채무회전기간     1.000075
재고자산회전기간     1.000075
매출채권회전기간     1.000075
1회전기간        1.000075
운전가본회전률      1.000075
순운전자본회전률     1.000075
유형자산회전율      1.000075
유동자산회전률      1.000075
당좌자산회전률      1.000075
부가가치(백만원)    1.000075
총자본투자효율      1.000075
설비투자효율       1.000075
기계투자효율       1.000075
부가가치율        1.000075
노동소득분배율      1.000075
자본분배율     

## 데이터 수집 및 전처리
### kosdaq

In [235]:
# 코스피 데이터 로드
kospi1 = pd.read_csv("./코스피데이터(연결재무제표).csv", encoding = "euc-kr")
kospi2 = pd.read_csv("./코스피데이터(개별재무제표).csv", encoding = "euc-kr")

# 컬럼명 변경
kospi1.columns = ["회사명", "거래소코드", "회계년도", "매출액증가율", "총자본증가율", "자기자본증가율", "순이익증가율", "유형자산증가율", "비유동자산증가율", "유동자산증가율", "재고자산증가율", "유동비율", "당좌비율", "현금비율", "순운전자본비율", "부채비율", "자기자본배율", "비유동비율", "유동자산구성비율", "자기자본구성비율", "매출액총이익률", "매출액순이익률", "총자본사업이익률", "총자본순이익률", "자기자본순이익률", "1주당순이익(원)", "1주당매출액(원)", "총자본회전률", "자기자본회전률", "비유동자산회전률", "재고자산회전률", "매출채권회전률", "매입채무회전률", "매입채무회전기간", "재고자산회전기간", "매출채권회전기간", "1회전기간", "운전가본회전률", "순운전자본회전률", "유형자산회전율", "유동자산회전률", "당좌자산회전률", "부가가치(백만원)", "총자본투자효율", "설비투자효율", "기계투자효율", "부가가치율", "노동소득분배율", "자본분배율", "이윤분배율", "상장폐지일", "소속코드", "상장일"]
kospi2.columns = ["회사명", "거래소코드", "회계년도", "매출액증가율", "총자본증가율", "자기자본증가율", "순이익증가율", "유형자산증가율", "비유동자산증가율", "유동자산증가율", "재고자산증가율", "유동비율", "당좌비율", "현금비율", "순운전자본비율", "부채비율", "자기자본배율", "비유동비율", "유동자산구성비율", "자기자본구성비율", "매출액총이익률", "매출액순이익률", "총자본사업이익률", "총자본순이익률", "자기자본순이익률", "1주당순이익(원)", "1주당매출액(원)", "총자본회전률", "자기자본회전률", "비유동자산회전률", "재고자산회전률", "매출채권회전률", "매입채무회전률", "매입채무회전기간", "재고자산회전기간", "매출채권회전기간", "1회전기간", "운전가본회전률", "순운전자본회전률", "유형자산회전율", "유동자산회전률", "당좌자산회전률", "부가가치(백만원)", "총자본투자효율", "설비투자효율", "기계투자효율", "부가가치율", "노동소득분배율", "자본분배율", "이윤분배율", "상장폐지일", "소속코드", "상장일"]

## 결측치를 개별재무제표로 대체함
kospi = kospi1.fillna(kospi2)

## 해당년도 재무제표가 존재하지 않는 기업 해당년도 삭제
# kospi.isna().sum()
kospi.dropna(subset=["매출액증가율"], inplace = True)
# kospi.isna().sum()

In [236]:
# 상장일 이전에 회계년도 데이터 삭제
kospi['상장일'] = pd.to_datetime(kospi['상장일'])
kospi['회계년도'] = pd.to_datetime(kospi['회계년도'], format='%Y/%m')
kospi = kospi.loc[kospi['회계년도'] >= kospi['상장일']]

In [237]:
kospi

,회사명,거래소코드,회계년도,매출액증가율,총자본증가율,자기자본증가율,순이익증가율,유형자산증가율,비유동자산증가율,유동자산증가율,...,총자본투자효율,설비투자효율,기계투자효율,부가가치율,노동소득분배율,자본분배율,이윤분배율,상장폐지일,소속코드,상장일
0,(주)DB하이텍,990,2018-12-01,-1.54,7.30,22.51,-21.19,7.56,6.69,8.40,...,28.98,61.13,1.409500e+02,46.18,42.92,57.08,28.09,NaN,1,1975-12-12
1,(주)DB하이텍,990,2019-12-01,20.64,2.10,15.45,20.51,3.18,-3.00,11.19,...,34.83,67.30,1.352100e+02,46.97,38.21,61.79,27.59,NaN,1,1975-12-12
2,(주)DB하이텍,990,2020-12-01,15.92,8.24,23.26,58.67,9.42,8.56,7.73,...,41.06,83.62,1.773200e+02,51.71,32.58,67.42,34.31,NaN,1,1975-12-12
3,(주)DB하이텍,990,2021-12-01,29.78,31.51,39.20,90.87,-2.50,2.14,77.51,...,48.06,134.52,2.847200e+02,61.33,24.03,75.97,42.54,NaN,1,1975-12-12
4,(주)DB하이텍,990,2022-12-01,37.92,36.95,48.12,76.35,12.18,8.19,62.88,...,50.35,189.56,3.855200e+02,63.80,20.01,79.99,52.29,NaN,1,1975-12-12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3688,흥아해운(주),3280,2018-12-01,-9.87,-5.33,-37.42,0.00,-12.07,-5.17,-6.33,...,4.24,6.91,1.135163e+09,4.52,201.90,-101.90,-254.12,NaN,1,1976-06-29
3689,흥아해운(주),3280,2019-12-01,-86.45,-46.38,-57.73,0.00,-51.51,-51.51,-13.42,...,3.10,5.59,4.455416e+08,13.08,241.39,-141.39,-384.23,NaN,1,1976-06-29
3690,흥아해운(주),3280,2020-12-01,-12.95,-33.46,-185.01,0.00,-24.76,-32.47,-37.01,...,-0.39,-0.62,0.000000e+00,-1.25,0.00,0.00,0.00,NaN,1,1976-06-29
3691,흥아해운(주),3280,2021-12-01,-8.11,-13.77,0.00,0.00,-20.91,-15.66,-6.48,...,22.88,39.81,0.000000e+00,69.25,41.38,58.62,29.04,NaN,1,1976-06-29


In [238]:
len(kospi[kospi['거래소코드'] > 900000])

5

In [239]:
## kospi 외국계 기업 종목코드 제거
kospi = kospi[kospi['거래소코드'] < 900000]

In [240]:
kospi

,회사명,거래소코드,회계년도,매출액증가율,총자본증가율,자기자본증가율,순이익증가율,유형자산증가율,비유동자산증가율,유동자산증가율,...,총자본투자효율,설비투자효율,기계투자효율,부가가치율,노동소득분배율,자본분배율,이윤분배율,상장폐지일,소속코드,상장일
0,(주)DB하이텍,990,2018-12-01,-1.54,7.30,22.51,-21.19,7.56,6.69,8.40,...,28.98,61.13,1.409500e+02,46.18,42.92,57.08,28.09,NaN,1,1975-12-12
1,(주)DB하이텍,990,2019-12-01,20.64,2.10,15.45,20.51,3.18,-3.00,11.19,...,34.83,67.30,1.352100e+02,46.97,38.21,61.79,27.59,NaN,1,1975-12-12
2,(주)DB하이텍,990,2020-12-01,15.92,8.24,23.26,58.67,9.42,8.56,7.73,...,41.06,83.62,1.773200e+02,51.71,32.58,67.42,34.31,NaN,1,1975-12-12
3,(주)DB하이텍,990,2021-12-01,29.78,31.51,39.20,90.87,-2.50,2.14,77.51,...,48.06,134.52,2.847200e+02,61.33,24.03,75.97,42.54,NaN,1,1975-12-12
4,(주)DB하이텍,990,2022-12-01,37.92,36.95,48.12,76.35,12.18,8.19,62.88,...,50.35,189.56,3.855200e+02,63.80,20.01,79.99,52.29,NaN,1,1975-12-12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3688,흥아해운(주),3280,2018-12-01,-9.87,-5.33,-37.42,0.00,-12.07,-5.17,-6.33,...,4.24,6.91,1.135163e+09,4.52,201.90,-101.90,-254.12,NaN,1,1976-06-29
3689,흥아해운(주),3280,2019-12-01,-86.45,-46.38,-57.73,0.00,-51.51,-51.51,-13.42,...,3.10,5.59,4.455416e+08,13.08,241.39,-141.39,-384.23,NaN,1,1976-06-29
3690,흥아해운(주),3280,2020-12-01,-12.95,-33.46,-185.01,0.00,-24.76,-32.47,-37.01,...,-0.39,-0.62,0.000000e+00,-1.25,0.00,0.00,0.00,NaN,1,1976-06-29
3691,흥아해운(주),3280,2021-12-01,-8.11,-13.77,0.00,0.00,-20.91,-15.66,-6.48,...,22.88,39.81,0.000000e+00,69.25,41.38,58.62,29.04,NaN,1,1976-06-29


In [241]:
## 증가율 0 인 데이터 평균값으로 대체
kospi['자기자본증가율'] = kospi['자기자본증가율'].replace(0, np.nan)
mean_growth_rate = kospi['자기자본증가율'].mean()
kospi['자기자본증가율'] = kospi['자기자본증가율'].fillna(mean_growth_rate)
kospi['순이익증가율'] = kospi['순이익증가율'].replace(0, np.nan)
mean_growth_rate = kospi['순이익증가율'].mean()
kospi['순이익증가율'] = kospi['순이익증가율'].fillna(mean_growth_rate)
kospi['유동자산증가율'] = kospi['유동자산증가율'].replace(0, np.nan)
mean_growth_rate = kospi['유동자산증가율'].mean()
kospi['유동자산증가율'] = kospi['유동자산증가율'].fillna(mean_growth_rate)
kospi['재고자산증가율'] = kospi['재고자산증가율'].replace(0, np.nan)
mean_growth_rate = kospi['재고자산증가율'].mean()
kospi['재고자산증가율'] = kospi['재고자산증가율'].fillna(mean_growth_rate)
kospi['유형자산증가율'] = kospi['유형자산증가율'].replace(0, np.nan)
mean_growth_rate = kospi['유형자산증가율'].mean()
kospi['유형자산증가율'] = kospi['유형자산증가율'].fillna(mean_growth_rate)
kospi['비유동자산증가율'] = kospi['비유동자산증가율'].replace(0, np.nan)
mean_growth_rate = kospi['비유동자산증가율'].mean()
kospi['비유동자산증가율'] = kospi['비유동자산증가율'].fillna(mean_growth_rate)
kospi['매출액증가율'] = kospi['매출액증가율'].replace(0, np.nan)
mean_growth_rate = kospi['매출액증가율'].mean()
kospi['매출액증가율'] = kospi['매출액증가율'].fillna(mean_growth_rate)
kospi['총자본증가율'] = kospi['총자본증가율'].replace(0, np.nan)
mean_growth_rate = kospi['총자본증가율'].mean()
kospi['총자본증가율'] = kospi['총자본증가율'].fillna(mean_growth_rate)

C:\Users\duja1\AppData\Local\Temp\ipykernel_1648\950300328.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  kospi['자기자본증가율'] = kospi['자기자본증가율'].replace(0, np.nan)
C:\Users\duja1\AppData\Local\Temp\ipykernel_1648\950300328.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  kospi['자기자본증가율'] = kospi['자기자본증가율'].fillna(mean_growth_rate)
C:\Users\duja1\AppData\Local\Temp\ipykernel_1648\950300328.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[ro

In [242]:
# 거래소코드와 소속코드 문자열로 변환
kospi = kospi.astype({"거래소코드" : "str"})
kospi = kospi.astype({"소속코드" : "str"})

In [243]:
# 이상치 확인
kospi['매출액증가율']

q5 = np.percentile(kospi['매출액증가율'], 5)
q95 = np.percentile(kospi['매출액증가율'], 95)

outliers = kospi['매출액증가율'][(kospi['매출액증가율'] < q5) | (kospi['매출액증가율'] > q95)]

print("하위 5%:", q5)
print("상위 5%:", q95)
print("이상치:", outliers)

하위 5%: -23.244
상위 5%: 48.906
이상치: 5       -30.76
16      -23.28
17      -33.56
18       49.39
23       67.55
         ...  
3679     78.12
3681     66.54
3684     62.30
3689    -86.45
3692    117.63
Name: 매출액증가율, Length: 360, dtype: float64


In [244]:
# 윈저라이징 실행(이상치 제거)
for i in range(3, 50, 1):
    kospi[kospi.columns[i]] = windsorize(kospi[kospi.columns[i]])

In [245]:
# 윈저라이징 실행 후 이상치 확인
kospi['매출액증가율']

q5 = np.percentile(kospi['매출액증가율'], 5)
q95 = np.percentile(kospi['매출액증가율'], 95)

outliers = kospi['매출액증가율'][(kospi['매출액증가율'] < q5) | (kospi['매출액증가율'] > q95)]

print("하위 5%:", q5)
print("상위 5%:", q95)
print("이상치:", outliers)

하위 5%: -23.2224
상위 5%: 48.9036
이상치: 5      -23.244
16     -23.244
17     -23.244
18      48.906
23      48.906
         ...  
3679    48.906
3681    48.906
3684    48.906
3689   -23.244
3692    48.906
Name: 매출액증가율, Length: 360, dtype: float64


In [246]:
# standard 스케일링
numeric_cols = kospi.select_dtypes(include=np.number).columns
scaler = StandardScaler()
kospi[numeric_cols] = scaler.fit_transform(kospi[numeric_cols])

print(kospi.head())

        회사명 거래소코드       회계년도    매출액증가율    총자본증가율   자기자본증가율    순이익증가율   
0  (주)DB하이텍   990 2018-12-01 -0.545082  0.029407  1.056604 -0.950430  \
1  (주)DB하이텍   990 2019-12-01  0.696357 -0.393348  0.580566 -0.662053   
2  (주)DB하이텍   990 2020-12-01  0.432174  0.105828  1.107175 -0.398156   
3  (주)DB하이텍   990 2021-12-01  1.207933  1.997657  2.181970 -0.175476   
4  (주)DB하이텍   990 2022-12-01  1.663538  2.439924  2.677024 -0.275890   

    유형자산증가율  비유동자산증가율   유동자산증가율  ...   총자본투자효율    설비투자효율    기계투자효율     부가가치율   
0  0.106446 -0.064104 -0.024497  ...  1.229743 -0.307570 -0.359919  1.901851  \
1 -0.150118 -0.690984  0.115852  ...  1.846057 -0.249279 -0.360986  1.964812   
2  0.215398  0.056872 -0.058201  ...  2.180658 -0.095095 -0.353156  2.255392   
3 -0.482831 -0.358459  2.395941  ...  2.180658  0.385785 -0.333188  2.255392   
4  0.377069  0.032936  2.395941  ...  2.180658  0.905778 -0.314447  2.255392   

    노동소득분배율     자본분배율     이윤분배율  상장폐지일  소속코드        상장일  
0 -0.514174  0.743839  0.629

### 상장폐지 데이터 전처리

In [247]:
# kospi 상장폐지 기업 추출
kospi_sp = kospi.loc[~(kospi["상장폐지일"].isna())]
kospi_sp["회사명"].unique()

array(['(주)동성화학', '(주)부산도시가스', '(주)사조해표', '(주)신한', '(주)엔에스쇼핑',
       '(주)폴루스바이오팜', '(주)한화갤러리아타임월드', '고려개발(주)', '두산건설(주)', '롯데푸드(주)',
       '알보젠코리아(주)', '에스비에스미디어홀딩스(주)', '웅진에너지(주)', '코리아오토글라스(주)',
       '코오롱머티리얼(주)', '한국제지(주)'], dtype=object)

In [248]:
# kosdaq 상장폐지 기업 추출
kosdaq_sp = kosdaq.loc[~(kosdaq["상장폐지일"].isna())]
kosdaq_sp["회사명"].unique()

array(['(주)금빛', '(주)녹십자셀', '(주)뉴로스', '(주)데코앤이', '(주)드림티엔터테인먼트', '(주)레드로버',
       '(주)맘스터치앤컴퍼니', '(주)맥스로텍', '(주)모다', '(주)미래SCI', '(주)미래테크놀로지',
       '(주)바이오빌', '(주)세영디앤씨', '(주)소리바다', '(주)썬테크놀로지스', '(주)씨엠에스에듀',
       '(주)아이엠텍', '(주)에스마크', '(주)에스에이티', '(주)에스에프씨', '(주)에스제이케이',
       '(주)에이치디', '(주)에이팸', '(주)에프티이앤이', '(주)엘엑스세미콘', '(주)엘피케이로보틱스',
       '(주)엠씨넥스', '(주)연이비앤티', '(주)원익피앤이', '(주)이매진아시아', '(주)이십일스토어',
       '(주)이엠네트웍스', '(주)인터파크', '(주)제낙스', '(주)제이콘텐트리', '(주)제이테크놀로지',
       '(주)지에스홈쇼핑', '(주)지유온', '(주)카이노스메드', '(주)켐온', '(주)코너스톤네트웍스',
       '(주)코리아센터', '(주)쿠첸', '(주)티씨엠생명과학', '(주)티에스트릴리온', '(주)파인넥스',
       '(주)파티게임즈', '(주)팍스넷', '(주)포스코케미칼', '(주)포인트엔지니어링', '(주)폭스소프트',
       '(주)퓨전', '(주)피앤텔', '(주)한국비엔씨', '(주)한국아트라스비엑스', '(주)한컴지엠디', '(주)한프',
       '(주)행남사', '(주)화신테크', '(주)화진', '넥슨지티(주)', '다이노나(주)', '디에스티(주)',
       '럭슬(주)', '미래오토스', '에스에이치엔엘(주)', '에스지에이클라우드서비스(주)', '에스케이 머티리얼즈(주)',
       '에스케이씨솔믹스(주)', '에이아이비트(주)', '원텍(주)', '이엘케이(주)', '자안바이오(주)',
       '지스마트글로벌(주)', '참존글로벌(

#### krx 상장폐지 현황 데이터의 상장폐지사유와 비교하여 부실여부 판단

In [249]:
# krx에서 20180101 ~ 20221231 상장폐지종목 데이터 수집 후 로드
sp = pd.read_excel("./상장폐지기업.xlsx")
cond = (sp["증권구분"] == "주권") & (sp["주식종류"] == "보통주") & (sp['시장구분'] != 'KONEX')
sp = sp[cond]
sp = sp.drop('번호', axis=1)
sp.to_excel("delist.xlsx")

c:\Users\duja1\AppData\Local\Programs\Python\Python310\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [250]:
# krx 데이터에서 kospi 상장폐지 종목 확인
sp.loc[sp['종목명'].isin(['동성화학', '부산도시가스', '사조해표', '신한', '엔에스쇼핑',
       '폴루스바이오팜', '한화갤러리아타임월드', '고려개발', '두산건설', '롯데푸드',
       '알보젠코리아', '에스비에스미디어홀딩스', '웅진에너지', '코리아오토글라스',
       '코오롱머티리얼', '한국제지', 'SBS미디어홀딩스', '부산가스'])]

,종목코드,종목명,시장구분,증권구분,주식종류,상장일,폐지일,폐지사유,정리매매기간_시작일,정리매매기간_종료일,업종,액면가,상장주식수,이전상장후_종목코드,이전상장후_종목명
321,002270,롯데푸드,KOSPI,주권,보통주,1973/05/09,2022/07/20,해산 사유 발생,NaN,NaN,음식료품,5000,1209055,NaN,NaN
359,138250,엔에스쇼핑,KOSPI,주권,보통주,2015/03/27,2022/03/22,지주회사(최대주주등)의 완전자회사화 등,NaN,NaN,유통업,500,33696000,NaN,NaN
363,007630,폴루스바이오팜,KOSPI,주권,보통주,1988/10/24,2022/02/22,감사의견 의견거절 2년 계속,2022/02/11,2022/02/21,유통업,500,45007695,NaN,NaN
369,101060,SBS미디어홀딩스,KOSPI,주권,보통주,2008/03/24,2022/01/18,해산 사유 발생,NaN,NaN,서비스업,500,139896190,NaN,NaN
370,015350,부산가스,KOSPI,주권,보통주,1997/06/23,2022/01/18,지주회사(최대주주등)의 완전자회사화 등,NaN,NaN,전기가스업,5000,11000000,NaN,NaN
382,144620,코오롱머티리얼,KOSPI,주권,보통주,2012/04/05,2021/11/05,지주회사(최대주주등)의 완전자회사화 등,NaN,NaN,화학,1000,68658635,NaN,NaN
427,005190,동성화학,KOSPI,주권,보통주,1988/07/01,2021/04/16,해산 사유 발생,NaN,NaN,화학,1000,5335564,NaN,NaN
448,152330,코리아오토글라스,KOSPI,주권,보통주,2015/12/29,2020/12/18,해산 사유 발생,NaN,NaN,비금속광물,5000,20000000,NaN,NaN
462,004200,고려개발,KOSPI,주권,보통주,1978/10/30,2020/07/21,해산 사유 발생,NaN,NaN,건설업,5000,15219444,NaN,NaN
464,002300,한국제지,KOSPI,주권,보통주,1971/06/23,2020/07/13,해산 사유 발생,NaN,NaN,종이목재,5000,5004949,NaN,NaN


In [251]:
# kospi 중 부실, 정상 기업 추출
kospi_bs = kospi.loc[kospi['회사명'].isin(['(주)동성화학', '(주)사조해표', '(주)신한', '(주)폴루스바이오팜', 
                                        '고려개발(주)', '롯데푸드(주)', '에스비에스미디어홀딩스(주)', 
                                        '웅진에너지(주)', '코리아오토글라스(주)', '한국제지(주)'])]
kospi_js = kospi.loc[~kospi['회사명'].isin(['(주)동성화학', '(주)사조해표', '(주)신한', '(주)폴루스바이오팜', 
                                         '고려개발(주)', '롯데푸드(주)', '에스비에스미디어홀딩스(주)', 
                                         '웅진에너지(주)', '코리아오토글라스(주)', '한국제지(주)'])]

In [252]:
# krx 데이터에서 kosdaq 상장폐지 종목 확인
sp.loc[sp['종목명'].isin(['금빛', '녹십자셀', '뉴로스', '데코앤이', '드림티엔터테인먼트', '레드로버',
       '맘스터치앤컴퍼니', '맥스로텍', '모다', '미래SCI', '미래테크놀로지',
       '바이오빌', '세영디앤씨', '소리바다', '썬테크놀로지스', '씨엠에스에듀',
       '아이엠텍', '에스마크', '에스에이티', '에스에프씨', '에스제이케이',
       '에이치디', '에이팸', '에프티이앤이', '엘엑스세미콘', '엘피케이로보틱스',
       '엠씨넥스', '연이비앤티', '원익피앤이', '이매진아시아', '이십일스토어',
       '이엠네트웍스', '인터파크', '제낙스', '제이콘텐트리', '제이테크놀로지',
       '지에스홈쇼핑', '지유온', '카이노스메드', '켐온', '코너스톤네트웍스',
       '코리아센터', '쿠첸', '티씨엠생명과학', '티에스트릴리온', '파인넥스',
       '파티게임즈', '팍스넷', '포스코케미칼', '포인트엔지니어링', '폭스소프트',
       '퓨전', '피앤텔', '한국비엔씨', '한국아트라스비엑스', '한컴지엠디', '한프',
       '행남사', '화신테크', '화진', '넥슨지티', '다이노나', '디에스티',
       '럭슬', '미래오토스', '에스에이치엔엘', '에스지에이클라우드서비스', '에스케이 머티리얼즈',
       '에스케이씨솔믹스', '에이아이비트', '원텍', '이엘케이', '자안바이오',
       '지스마트글로벌', '참존글로벌', '케이맥', '피아이첨단소재', '한일네트웍스',
       '현대정보기술', '현진소재'])]

,종목코드,종목명,시장구분,증권구분,주식종류,상장일,폐지일,폐지사유,정리매매기간_시작일,정리매매기간_종료일,업종,액면가,상장주식수,이전상장후_종목코드,이전상장후_종목명
294,217600,켐온,KOSDAQ,주권,보통주,2015/06/26,2022/12/16,피흡수합병,NaN,NaN,기타서비스,100,63861340,NaN,NaN
295,290510,코리아센터,KOSDAQ,주권,보통주,2019/11/29,2022/12/16,피흡수합병,NaN,NaN,소프트웨어,100,114534446,NaN,NaN
297,131390,원익피앤이,KOSDAQ,주권,보통주,2011/09/27,2022/11/29,피흡수합병,NaN,NaN,일반전기전자,500,14818910,NaN,NaN
298,158310,참존글로벌,KOSDAQ,주권,보통주,2013/04/03,2022/11/09,기업의 계속성 및 경영의 투명성 등을 종합적으로 고려하여 상장폐지기준에 해당한다고 결정,2022/10/31,2022/11/08,기계·장비,500,47841128,NaN,NaN
299,060300,레드로버,KOSDAQ,주권,보통주,2002/01/10,2022/11/04,기업의 계속성 및 경영의 투명성 등을 종합적을 고려하여 상장폐지기준에 해당한다고 결정,2022/10/26,2022/11/03,오락·문화,500,29231297,NaN,NaN
301,126870,뉴로스,KOSDAQ,주권,보통주,2012/02/15,2022/10/26,감사의견 거절(감사범위 제한 및 계속기업 존속능력 불확실성),2022/10/17,2022/10/25,기계·장비,500,35639398,NaN,NaN
304,090740,연이비앤티,KOSDAQ,주권,보통주,2007/09/19,2022/10/17,기업의 계속성 및 경영의 투명성 등을 종합적으로 고려하여 상장폐지기준에 해당한다고 결정,2022/10/05,2022/10/14,IT부품,500,22459415,NaN,NaN
307,046110,한일네트웍스,KOSDAQ,주권,보통주,2006/01/20,2022/10/05,상장폐지 신청('22.08.31),2022/09/23,2022/10/04,컴퓨터서비스,500,11954227,NaN,NaN
312,053110,소리바다,KOSDAQ,주권,보통주,2001/10/09,2022/09/07,감사의견 거절(감사범위 제한),2022/08/29,2022/09/06,디지털컨텐츠,500,8940349,NaN,NaN
338,050320,에스에이치엔엘,KOSDAQ,주권,보통주,2002/01/15,2022/05/31,감사의견 거절,2022/05/20,2022/05/30,금속,100,65661760,NaN,NaN


In [253]:
# kosdaq 중 부실, 정상 기업 추출
kosdaq_bs = kosdaq.loc[kosdaq['회사명'].isin(['(주)썬테크놀로지스', '참존글로벌(주)', '(주)레드로버', '(주)뉴로스', '(주)연이비앤티', 
'(주)소리바다', '에스에이치엔엘(주)', '(주)세영디앤씨', '현진소재(주)', '(주)한프',
'지스마트글로벌(주)', '자안바이오(주)', '(주)이엠네트웍스', '(주)코너스톤네트웍스', 
'에이아이비트(주)', '(주)지유온', '(주)맥스로텍', '(주)에스제이케이', '디에스티(주)', 
'(주)팍스넷', '럭슬(주)', '(주)제낙스', '(주)아이엠텍', '(주)퓨전', '(주)행남사',
'(주)에이팸', '(주)에이치디', '(주)미래SCI', '(주)이매진아시아', '(주)파티게임즈', 
'(주)바이오빌', '(주)모다', '(주)화진', '(주)에스에프씨', '이엘케이(주)', '(주)에스마크',
'(주)피앤텔', '(주)파인넥스', '(주)제이테크놀로지', '(주)데코앤이', '(주)에프티이앤이',
'(주)이십일스토어', '(주)화신테크', '미래오토스', '에스지에이클라우드서비스(주)'])]

kosdaq_js = kosdaq.loc[~kosdaq['회사명'].isin(['(주)썬테크놀로지스', '참존글로벌(주)', '(주)레드로버', '(주)뉴로스', '(주)연이비앤티', 
'(주)소리바다', '에스에이치엔엘(주)', '(주)세영디앤씨', '현진소재(주)', '(주)한프',
'지스마트글로벌(주)', '자안바이오(주)', '(주)이엠네트웍스', '(주)코너스톤네트웍스', 
'에이아이비트(주)', '(주)지유온', '(주)맥스로텍', '(주)에스제이케이', '디에스티(주)', 
'(주)팍스넷', '럭슬(주)', '(주)제낙스', '(주)아이엠텍', '(주)퓨전', '(주)행남사',
'(주)에이팸', '(주)에이치디', '(주)미래SCI', '(주)이매진아시아', '(주)파티게임즈', 
'(주)바이오빌', '(주)모다', '(주)화진', '(주)에스에프씨', '이엘케이(주)', '(주)에스마크',
'(주)피앤텔', '(주)파인넥스', '(주)제이테크놀로지', '(주)데코앤이', '(주)에프티이앤이',
'(주)이십일스토어', '(주)화신테크', '미래오토스', '에스지에이클라우드서비스(주)'])]

In [254]:
# kospi 데이터에 "부실여부" 파생변수 삽입
kospibs = '(주)동성화학', '(주)사조해표', '(주)신한', '(주)폴루스바이오팜', '고려개발(주)', '롯데푸드(주)', '에스비에스미디어홀딩스(주)', '웅진에너지(주)', '코리아오토글라스(주)', '한국제지(주)'
kospi["부실여부"] = kospi['회사명'].apply(lambda x: 1 if x in kospibs else 0)
kospi.head()

,회사명,거래소코드,회계년도,매출액증가율,총자본증가율,자기자본증가율,순이익증가율,유형자산증가율,비유동자산증가율,유동자산증가율,...,설비투자효율,기계투자효율,부가가치율,노동소득분배율,자본분배율,이윤분배율,상장폐지일,소속코드,상장일,부실여부
0,(주)DB하이텍,990,2018-12-01,-0.545082,0.029407,1.056604,-0.950430,0.106446,-0.064104,-0.024497,...,-0.307570,-0.359919,1.901851,-0.514174,0.743839,0.629616,NaN,1,1975-12-12,0
1,(주)DB하이텍,990,2019-12-01,0.696357,-0.393348,0.580566,-0.662053,-0.150118,-0.690984,0.115852,...,-0.249279,-0.360986,1.964812,-0.681920,0.923410,0.613246,NaN,1,1975-12-12,0
2,(주)DB하이텍,990,2020-12-01,0.432174,0.105828,1.107175,-0.398156,0.215398,0.056872,-0.058201,...,-0.095095,-0.353156,2.255392,-0.882431,1.138056,0.833252,NaN,1,1975-12-12,0
3,(주)DB하이텍,990,2021-12-01,1.207933,1.997657,2.181970,-0.175476,-0.482831,-0.358459,2.395941,...,0.385785,-0.333188,2.255392,-1.186938,1.464028,1.102692,NaN,1,1975-12-12,0
4,(주)DB하이텍,990,2022-12-01,1.663538,2.439924,2.677024,-0.275890,0.377069,0.032936,2.395941,...,0.905778,-0.314447,2.255392,-1.330109,1.468603,1.404545,NaN,1,1975-12-12,0


In [255]:
# kosdaq 데이터에 "부실여부" 파생변수 삽입
kosdaqbs = '(주)썬테크놀로지스', '참존글로벌(주)', '(주)레드로버', '(주)뉴로스', '(주)연이비앤티', 
'(주)소리바다', '에스에이치엔엘(주)', '(주)세영디앤씨', '현진소재(주)', '(주)한프',
'지스마트글로벌(주)', '자안바이오(주)', '(주)이엠네트웍스', '(주)코너스톤네트웍스', 
'에이아이비트(주)', '(주)지유온', '(주)맥스로텍', '(주)에스제이케이', '디에스티(주)', 
'(주)팍스넷', '럭슬(주)', '(주)제낙스', '(주)아이엠텍', '(주)퓨전', '(주)행남사',
'(주)에이팸', '(주)에이치디', '(주)미래SCI', '(주)이매진아시아', '(주)파티게임즈', 
'(주)바이오빌', '(주)모다', '(주)화진', '(주)에스에프씨', '이엘케이(주)', '(주)에스마크',
'(주)피앤텔', '(주)파인넥스', '(주)제이테크놀로지', '(주)데코앤이', '(주)에프티이앤이',
'(주)이십일스토어', '(주)화신테크', '미래오토스', '에스지에이클라우드서비스(주)'
kosdaq["부실여부"] = kosdaq['회사명'].apply(lambda x: 1 if x in kosdaqbs else 0)
kosdaq.head()

,회사명,거래소코드,회계년도,매출액증가율,총자본증가율,자기자본증가율,순이익증가율,유형자산증가율,비유동자산증가율,유동자산증가율,...,설비투자효율,기계투자효율,부가가치율,노동소득분배율,자본분배율,이윤분배율,상장폐지일,소속코드,상장일,부실여부
0,(주)CMG제약,58820,2018-12-01,0.267103,-0.329883,-0.251115,1.124574,-0.273520,0.036829,-0.406337,...,-0.176940,-0.346626,0.465836,0.052136,0.247429,0.637277,NaN,5,2001-08-31,0
1,(주)CMG제약,58820,2019-12-01,0.182738,2.461924,2.084707,-1.427800,0.606500,0.344726,2.120575,...,-0.148162,-0.352061,0.421143,0.151421,0.134455,0.331270,NaN,5,2001-08-31,0
2,(주)CMG제약,58820,2020-12-01,-0.074763,-0.451324,-0.446753,-1.983355,0.169694,0.101700,-0.502793,...,-0.170151,-0.347129,0.215627,0.406509,-0.155806,0.104944,NaN,5,2001-08-31,0
3,(주)CMG제약,58820,2021-12-01,-0.148625,-0.436856,-0.396974,2.546591,-0.382989,1.074120,-0.829672,...,-0.105667,-0.330288,0.262012,0.241663,0.031770,0.337093,NaN,5,2001-08-31,0
4,(주)CMG제약,58820,2022-12-01,0.208149,-0.212389,-0.460678,0.294699,0.155810,-0.291319,-0.184250,...,-0.174424,-0.338809,-0.022055,0.747506,-0.543821,0.048248,NaN,5,2001-08-31,0


In [256]:
# kospi, kosdaq 데이터 병합
data = pd.concat([kospi, kosdaq], axis = 0)

## 피쳐선정

In [257]:
# lasso

data_feature = data[["매출액증가율", "총자본증가율", "자기자본증가율", "순이익증가율", "유형자산증가율", "비유동자산증가율", 
          "유동자산증가율", "재고자산증가율", "유동비율", "당좌비율", "현금비율", "순운전자본비율", "부채비율", 
          "자기자본배율", "비유동비율", "유동자산구성비율", "자기자본구성비율", "매출액총이익률", "매출액순이익률", 
          "총자본사업이익률", "총자본순이익률", "자기자본순이익률", "1주당순이익(원)", "1주당매출액(원)", "총자본회전률", 
          "자기자본회전률", "비유동자산회전률", "재고자산회전률", "매출채권회전률", "매입채무회전률", "매입채무회전기간", 
          "재고자산회전기간", "매출채권회전기간", "1회전기간", "운전가본회전률", "순운전자본회전률", "유형자산회전율", 
          "유동자산회전률", "당좌자산회전률", "부가가치(백만원)", "총자본투자효율", "설비투자효율", "기계투자효율", 
          "부가가치율", "노동소득분배율", "자본분배율", "이윤분배율"]]

feature = data_feature
target = data['부실여부']

lasso = SelectFromModel(LogisticRegression(C=0.2, penalty='l1',solver='liblinear'))
lasso.fit(feature, target)
lasso_support = lasso.get_support()
lr_feature = feature.loc[:,lasso_support].columns.tolist()
lr_feature

['매출액증가율',
 '자기자본증가율',
 '순이익증가율',
 '유형자산증가율',
 '비유동자산증가율',
 '재고자산증가율',
 '순운전자본비율',
 '매출액순이익률',
 '총자본사업이익률',
 '1주당매출액(원)',
 '재고자산회전률',
 '매출채권회전률',
 '매입채무회전률',
 '매입채무회전기간',
 '매출채권회전기간',
 '1회전기간',
 '운전가본회전률',
 '유동자산회전률',
 '부가가치(백만원)',
 '설비투자효율',
 '부가가치율',
 '노동소득분배율',
 '자본분배율']

In [258]:
# selectKBest

data_feature = data[["매출액증가율", "총자본증가율", "자기자본증가율", "순이익증가율", "유형자산증가율", "비유동자산증가율", 
          "유동자산증가율", "재고자산증가율", "유동비율", "당좌비율", "현금비율", "순운전자본비율", "부채비율", 
          "자기자본배율", "비유동비율", "유동자산구성비율", "자기자본구성비율", "매출액총이익률", "매출액순이익률", 
          "총자본사업이익률", "총자본순이익률", "자기자본순이익률", "1주당순이익(원)", "1주당매출액(원)", "총자본회전률", 
          "자기자본회전률", "비유동자산회전률", "재고자산회전률", "매출채권회전률", "매입채무회전률", "매입채무회전기간", 
          "재고자산회전기간", "매출채권회전기간", "1회전기간", "운전가본회전률", "순운전자본회전률", "유형자산회전율", 
          "유동자산회전률", "당좌자산회전률", "부가가치(백만원)", "총자본투자효율", "설비투자효율", "기계투자효율", 
          "부가가치율", "노동소득분배율", "자본분배율", "이윤분배율"]]

# 독립변수와 종속변수를 분리합니다.
X = data_feature
y = data['부실여부']

# f 검정 방법을 사용해 피처를 선택합니다.'k=선정개수'
selector = SelectKBest(score_func=f_classif, k=20)
X_new = selector.fit_transform(X, y)

# 선택된 피처를 출력합니다.
print(X.columns[selector.get_support()])

Index(['매출액증가율', '총자본증가율', '유형자산증가율', '비유동자산증가율', '현금비율', '순운전자본비율',
       '자기자본구성비율', '매출액순이익률', '총자본사업이익률', '총자본순이익률', '자기자본순이익률', '1주당순이익(원)',
       '매입채무회전기간', '매출채권회전기간', '순운전자본회전률', '부가가치(백만원)', '총자본투자효율', '부가가치율',
       '노동소득분배율', '자본분배율'],
      dtype='object')


## 모델링 및 성능평가

In [259]:
# 랜덤포레스트

# 학습용 데이터와 테스트용 데이터 분할
train = data[data['회계년도'] < '2021-01-01']
test = data[data['회계년도'] >= '2021-01-01']

# 학습용 데이터 전처리
X_train = train[["현금비율", "순운전자본비율", '매출액순이익률', "총자본사업이익률", "부가가치율"]]
y_train = train['부실여부']

# 테스트용 데이터 전처리
X_test = test[["현금비율", "순운전자본비율", '매출액순이익률', "총자본사업이익률", "부가가치율"]]
y_test = test['부실여부']

# 랜덤 포레스트 모델 학습
rf = RandomForestRegressor(n_estimators=100, max_depth=5, random_state=1)
rf.fit(X_train, y_train)

# 테스트 데이터에 대한 예측
y_pred = rf.predict(X_test)

# 정확도 계산 및 출력
accuracy = accuracy_score(y_test, y_pred.round())
print('Accuracy:', accuracy)

Accuracy: 0.9986136783733827


In [260]:
# 로지스틱

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# 모델 선택 및 훈련
model = LogisticRegression()
model.fit(X_train, y_train)

# 모델 평가
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.9988447319778189


## 지수화
### 코스피200 대형주 50% 그 외 50% 랜덤추출하여 60종목 편입

In [261]:
# 코스피200 1분기, 반기, 기업규모 데이터 불러오기
df1 = pd.read_excel("./2020코스피200_1분기.xlsx")
df2 = pd.read_excel("./2020코스피200_반기.xlsx")
Scale = pd.read_csv("./기업규모코드.csv", encoding = "euc-kr")

In [262]:
# 데이터 프레임 크기 맞추기
common_companies = pd.Series(list(set(df1['회사명']) & set(df2['회사명']) & set(Scale['회사명'])))

df1 = df1[df1['회사명'].isin(common_companies)]
df2 = df2[df2['회사명'].isin(common_companies)]
Scale = Scale[Scale['회사명'].isin(common_companies)]

In [263]:
# 1분기 인덱스 리셋
df1 = df1.reset_index(drop=True)

In [264]:
# 반기 인덱스 리셋
df2 = df2.reset_index(drop=True)

In [265]:
# 1분기와 반기 데이터에 기업규모코드 추가
df1['기업규모코드'] = Scale['기업규모코드']
df2['기업규모코드'] = Scale['기업규모코드']

In [266]:
# 컬럼 이름 변경
df1.columns = ['회사명', '거래소코드', '회계년도', '자산', '현금및현금성자산', '부채', '유동자산', '유동부채', '원재료', '선급공사비,외주비', '매출액', '법인세비용차감전순이익', '법인세비용', '당기순이익', '기업규모코드']
df2.columns = ['회사명', '거래소코드', '회계년도', '자산', '현금및현금성자산', '부채', '유동자산', '유동부채', '원재료', '선급공사비,외주비', '매출액', '법인세비용차감전순이익', '법인세비용', '당기순이익', '기업규모코드']

In [267]:
# 컬럼 추출
df_1 = df1[['자산', '현금및현금성자산', '부채', '유동자산', '유동부채', '원재료', '선급공사비,외주비', '매출액', '법인세비용차감전순이익', '법인세비용', '당기순이익']]
df_2 = df2[['자산', '현금및현금성자산', '부채', '유동자산', '유동부채', '원재료', '선급공사비,외주비', '매출액', '법인세비용차감전순이익', '법인세비용', '당기순이익']]

In [268]:
# 2분기 데이터 추출(반기 - 1분기)
df_2 = df_2 - df_1

In [269]:
# 필요한 컬럼만 추출
d1 = pd.concat([df1[['회사명', '거래소코드', '회계년도', '기업규모코드']], df_1], axis=1)
d2 = pd.concat([df2[['회사명', '거래소코드', '회계년도', '기업규모코드']], df_2], axis=1)

In [270]:
# 셀렉된 피처 컬럼 생성(1분기)
# 분기보고서에 없는 파생변수 생성
d1['현금비율'] = (d1['현금및현금성자산'] / d1['부채']) * 100
d1['순운전자본비율'] = ((d1['유동자산'] - d1['유동부채']) / d1['매출액']) * 100
d1['매출액순이익률'] = ((d1['법인세비용차감전순이익'] - d1['법인세비용']) / d1['매출액']) * 100
d1['총자본사업이익률'] = (d1['당기순이익'] / (d1['자산'] - d1['부채'])) * 100
d1['부가가치율'] = ((d1['매출액'] - d1['원재료'] - d1['선급공사비,외주비']) / d1['매출액']) * 100

In [271]:
# 셀렉된 피처 컬럼 생성(2분기)
# 분기보고서에 없는 파생변수 생성
d2['현금비율'] = (d2['현금및현금성자산'] / d2['부채']) * 100
d2['순운전자본비율'] = ((d2['유동자산'] - d2['유동부채']) / d2['매출액']) * 100
d2['매출액순이익률'] = ((d2['법인세비용차감전순이익'] - d2['법인세비용']) / d2['매출액']) * 100
d2['총자본사업이익률'] = (d2['당기순이익'] / (d2['자산'] - d2['부채'])) * 100
d2['부가가치율'] = ((d2['매출액'] - d2['원재료'] - d2['선급공사비,외주비']) / d2['매출액']) * 100

In [272]:
# 필요한 컬럼만 추출
d1 = d1[['회사명', '거래소코드', '회계년도', '기업규모코드', "현금비율", "순운전자본비율", '매출액순이익률', "총자본사업이익률", "부가가치율"]]
d2 = d2[['회사명', '거래소코드', '회계년도', '기업규모코드', "현금비율", "순운전자본비율", '매출액순이익률', "총자본사업이익률", "부가가치율"]]

### 펀드에 편입할 종목 결정 후 추출

In [273]:
### 가입시점 펀드에 편압된 기업 추출 ###

# 대기업 30개 그 외 30개 기업 랜덤 추출
total_d1 = d1.dropna()

# 기업규모코드가 10인 데이터 추출
scale_10_ = total_d1.loc[total_d1['기업규모코드'] == 10]

# 기업규모코드가 10이 아닌 데이터 추출
scale_N10_ = total_d1.loc[total_d1['기업규모코드'] != 10]

# scale_10에서 랜덤하게 30개 추출
scale_10 = scale_10_.sample(n=30, random_state=12)

# scale_N10에서 랜덤하게 30개 추출
scale_N10 = scale_N10_.sample(n=30, random_state=12)

# 추출한 기업들을 하나로 합치기
selected_companies_1 = pd.concat([scale_10, scale_N10])

In [274]:
### 현재시점 펀드에 편압된 기업 추출 ###

# 대기업 30개 그 외 30개 기업 랜덤 추출
total_d2 = d2.dropna()

# 기업규모코드가 10인 데이터 추출
scale_10_ = total_d2.loc[total_d2['기업규모코드'] == 10]

# 기업규모코드가 10이 아닌 데이터 추출
scale_N10_ = total_d2.loc[total_d2['기업규모코드'] != 10]

# scale_10에서 랜덤하게 30개 추출
scale_10 = scale_10_.sample(n=30, random_state=20)

# scale_N10에서 랜덤하게 30개 추출
scale_N10 = scale_N10_.sample(n=30, random_state=45)

# 추출한 기업들을 하나로 합치기
selected_companies_2 = pd.concat([scale_10, scale_N10])

In [275]:
# 랜덤 추출된 기업 데이터 인덱스 리셋
selected_companies_1 = selected_companies_1.reset_index(drop=True)
selected_companies_2 = selected_companies_2.reset_index(drop=True)

In [276]:
# 결측치 제거후 인덱스 리셋
d1 = d1.dropna().reset_index(drop=True)
d2 = d2.dropna().reset_index(drop=True)

### 펀드의 내부위험지수 산출과정
#### 가입시점

In [277]:
# 10분할 점수 부여 기준
columns = d1.columns.tolist()[4:]

# 각 열에서 상위 10개의 데이터를 추출하는 for문을 작성합니다.
result = []
for col in columns:
    data = d1[col].sort_values(ascending=False).reset_index(drop=True).loc[[14, 30, 45, 60, 76, 91, 107, 122, 138, 153]]
    result.append(data)

# 추출한 결과를 데이터프레임으로 합쳐줍니다.
d1_top10 = pd.concat(result, axis=1)

# 데이터프레임을 출력합니다.
d1_top10


,현금비율,순운전자본비율,매출액순이익률,총자본사업이익률,부가가치율
14,42.669970,192.873323,13.207272,5.068999,100.000000
30,31.176770,140.669798,9.015971,3.454742,99.702030
45,21.404190,101.134585,7.046674,2.388860,97.680311
60,17.256386,79.626484,5.512844,1.971831,92.929144
76,14.434573,53.978369,3.686783,1.407884,90.544267
91,11.581141,31.368602,2.305742,1.017760,86.705140
107,8.319922,13.481653,0.451564,0.374696,83.595973
122,7.001685,-9.435525,-1.243232,-0.463690,79.727531
138,4.108714,-51.952946,-7.794690,-4.272414,73.089482
153,0.962602,-2871.530690,-1695.489900,-324.956135,-117.818673


In [278]:
# 현금비율의 등급
feature_1 = pd.DataFrame({
    '현금비율': [x for x in selected_companies_1['현금비율']]
})

# 등급 매핑을 위한 함수 생성
def get_grade(value):
    if value >= 42.669970:
        return 1
    elif value >= 31.176770:
        return 2
    elif value >= 21.404190:
        return 3
    elif value >= 17.256386:
        return 4
    elif value >= 14.434573:
        return 5
    elif value >= 11.581141:
        return 6
    elif value >= 8.319922:
        return 7
    elif value >= 7.001685:
        return 8
    elif value >= 4.108714:
        return 9
    else:
        return 10

# apply 함수를 사용하여 등급 컬럼 추가
feature_1['등급'] = feature_1['현금비율'].apply(get_grade)

grade1 = feature_1['등급'].sum()
grade1


310

In [279]:
# 순운전자본비율의 등급
feature_2 = pd.DataFrame({
    '순운전자본비율': [x for x in selected_companies_1['순운전자본비율']]
})

# 등급 매핑을 위한 함수 생성
def get_grade(value):
    if value >= 192.873323:
        return 1
    elif value >= 140.669798:
        return 2
    elif value >= 101.134585:
        return 3
    elif value >= 79.626484:
        return 4
    elif value >= 53.978369:
        return 5
    elif value >= 31.368602:
        return 6
    elif value >= 13.481653:
        return 7
    elif value >= -9.435525:
        return 8
    elif value >= -51.952946:
        return 9
    else:
        return 10

# apply 함수를 사용하여 등급 컬럼 추가
feature_2['등급'] = feature_2['순운전자본비율'].apply(get_grade)

grade2 = feature_2['등급'].sum()
grade2


305

In [280]:
# 매출액순이익률의 등급
feature_3 = pd.DataFrame({
    '매출액순이익률': [x for x in selected_companies_1['매출액순이익률']]
})

# 등급 매핑을 위한 함수 생성
def get_grade(value):
    if value >= 13.207272:
        return 1
    elif value >= 9.015971:
        return 2
    elif value >= 7.046674:
        return 3
    elif value >= 5.512844:
        return 4
    elif value >= 3.686783:
        return 5
    elif value >= 2.305742:
        return 6
    elif value >= 0.451564:
        return 7
    elif value >= -1.243232:
        return 8
    elif value >= -7.794690:
        return 9
    else:
        return 10

# apply 함수를 사용하여 등급 컬럼 추가
feature_3['등급'] = feature_3['매출액순이익률'].apply(get_grade)

# 결과 출력
grade3 = feature_3['등급'].sum()
grade3

302

In [281]:
# 총자본사업이익률의 등급
feature_4 = pd.DataFrame({
    '총자본사업이익률': [x for x in selected_companies_1['총자본사업이익률']]
})

# 등급 매핑을 위한 함수 생성
def get_grade(value):
    if value >= 5.068999:
        return 1
    elif value >= 3.454742:
        return 2
    elif value >= 2.388860:
        return 3
    elif value >= 1.971831:
        return 4
    elif value >= 1.407884:
        return 5
    elif value >= 1.017760:
        return 6
    elif value >= 0.374696:
        return 7
    elif value >= -0.463690:
        return 8
    elif value >= -4.272414:
        return 9
    else:
        return 10

# apply 함수를 사용하여 등급 컬럼 추가
feature_4['등급'] = feature_4['총자본사업이익률'].apply(get_grade)

grade4 = feature_4['등급'].sum()
grade4

314

In [282]:
# 부가가치율의 등급
feature_5 = pd.DataFrame({
    '부가가치율': [x for x in selected_companies_1['총자본사업이익률']]
})

# 등급 매핑을 위한 함수 생성
def get_grade(value):
    if value >= 100.000000:
        return 1
    elif value >= 99.702030:
        return 2
    elif value >= 97.680311:
        return 3
    elif value >= 92.929144:
        return 4
    elif value >= 90.544267:
        return 5
    elif value >= 86.705140:
        return 6
    elif value >= 83.595973:
        return 7
    elif value >= 79.727531:
        return 8
    elif value >= 73.089482:
        return 9
    else:
        return 10

# apply 함수를 사용하여 등급 컬럼 추가
feature_5['등급'] = feature_5['부가가치율'].apply(get_grade)

grade5 = feature_5['등급'].sum()
grade5

597

#### 현재시점

In [283]:
# 10분할 점수 부여 기준
columns = d2.columns.tolist()[4:]

# 각 열에서 상위 10개의 데이터를 추출하는 for문을 작성합니다.
result = []
for col in columns:
    data = d2[col].sort_values(ascending=False).reset_index(drop=True).loc[[14, 30, 45, 60, 76, 91, 107, 122, 138, 153]]
    result.append(data)

# 추출한 결과를 데이터프레임으로 합쳐줍니다.
d2_top10 = pd.concat(result, axis=1)

# 데이터프레임을 출력합니다.
d2_top10

,현금비율,순운전자본비율,매출액순이익률,총자본사업이익률,부가가치율
14,245.419328,30.738807,13.937082,141.331309,101.818218
30,116.962658,17.265843,9.174021,105.086744,100.597142
45,79.685660,9.865081,6.667611,100.238322,100.026696
60,49.161383,6.355522,5.197036,98.771828,100.000000
76,21.334342,2.676220,3.631935,93.060983,99.946618
91,5.250606,0.545803,2.637603,80.476820,99.521083
107,-15.049444,-2.275697,1.366735,59.841530,99.062290
122,-85.682368,-9.067939,-0.086659,26.455501,98.288158
138,-292.014915,-27.798076,-5.619149,-9.279052,97.494356
153,-9832.346071,-2843.537475,-2887.053813,-5884.794157,-11.051269


In [284]:
# 현금비율의 등급
feature1 = pd.DataFrame({
    '현금비율': [x for x in selected_companies_2['현금비율']]
})

# 등급 매핑을 위한 함수 생성
def get_grade(value):
    if value >= 245.419328:
        return 1
    elif value >= 116.962658:
        return 2
    elif value >= 79.685660:
        return 3
    elif value >= 49.161383:
        return 4
    elif value >= 21.334342:
        return 5
    elif value >= 5.250606:
        return 6
    elif value >= -15.049444:
        return 7
    elif value >= -85.682368:
        return 8
    elif value >= -292.014915:
        return 9
    else:
        return 10

# apply 함수를 사용하여 등급 컬럼 추가
feature1['등급'] = feature1['현금비율'].apply(get_grade)

grade_1 = feature1['등급'].sum()
grade_1


359

In [285]:
# 순운전자본비율의 등급
feature2 = pd.DataFrame({
    '순운전자본비율': [x for x in selected_companies_2['순운전자본비율']]
})

# 등급 매핑을 위한 함수 생성
def get_grade(value):
    if value >= 30.738807:
        return 1
    elif value >= 17.265843:
        return 2
    elif value >= 9.865081:
        return 3
    elif value >= 6.355522:
        return 4
    elif value >= 2.676220:
        return 5
    elif value >= 0.545803:
        return 6
    elif value >= -2.275697:
        return 7
    elif value >= -9.067939:
        return 8
    elif value >= -27.798076:
        return 9
    else:
        return 10

# apply 함수를 사용하여 등급 컬럼 추가
feature2['등급'] = feature2['순운전자본비율'].apply(get_grade)

grade_2 = feature2['등급'].sum()
grade_2


357

In [286]:
# 매출액순이익률의 등급
feature3 = pd.DataFrame({
    '매출액순이익률': [x for x in selected_companies_2['매출액순이익률']]
})

# 등급 매핑을 위한 함수 생성
def get_grade(value):
    if value >= 13.937082:
        return 1
    elif value >= 9.174021:
        return 2
    elif value >= 6.667611:
        return 3
    elif value >= 5.197036:
        return 4
    elif value >= 3.631935:
        return 5
    elif value >= 2.637603:
        return 6
    elif value >= 1.366735:
        return 7
    elif value >= -0.086659:
        return 8
    elif value >= -5.619149:
        return 9
    else:
        return 10

# apply 함수를 사용하여 등급 컬럼 추가
feature3['등급'] = feature3['매출액순이익률'].apply(get_grade)

# 결과 출력
grade_3 = feature3['등급'].sum()
grade_3

348

In [287]:
# 총자본사업이익률의 등급
feature4 = pd.DataFrame({
    '총자본사업이익률': [x for x in selected_companies_2['총자본사업이익률']]
})

# 등급 매핑을 위한 함수 생성
def get_grade(value):
    if value >= 141.331309:
        return 1
    elif value >= 105.086744:
        return 2
    elif value >= 100.238322:
        return 3
    elif value >= 98.771828:
        return 4
    elif value >= 93.060983:
        return 5
    elif value >= 80.476820:
        return 6
    elif value >= 59.841530:
        return 7
    elif value >= 26.455501:
        return 8
    elif value >= -9.279052:
        return 9
    else:
        return 10

# apply 함수를 사용하여 등급 컬럼 추가
feature4['등급'] = feature4['총자본사업이익률'].apply(get_grade)

grade_4 = feature4['등급'].sum()
grade_4

367

In [288]:
# 부가가치율의 등급
feature5 = pd.DataFrame({
    '부가가치율': [x for x in selected_companies_2['총자본사업이익률']]
})

# 등급 매핑을 위한 함수 생성
def get_grade(value):
    if value >= 101.818218:
        return 1
    elif value >= 100.597142:
        return 2
    elif value >= 100.026696:
        return 3
    elif value >= 100.000000:
        return 4
    elif value >= 99.946618:
        return 5
    elif value >= 99.521083:
        return 6
    elif value >= 99.062290:
        return 7
    elif value >= 98.288158:
        return 8
    elif value >= 97.494356:
        return 9
    else:
        return 10

# apply 함수를 사용하여 등급 컬럼 추가
feature5['등급'] = feature5['부가가치율'].apply(get_grade)

grade_5 = feature5['등급'].sum()
grade_5

457

### 가입시점 펀드위험내부지수 : 30.0

In [289]:
join = (grade1 + grade2 + grade3 + grade4 + grade5) * 50/3000
join.round()

30.0

### 현재시점 펀드위험내부지수 : 31.0

In [290]:
present = (grade_1 + grade_2 + grade_3 + grade_4 + grade_5) * 50/3000
present.round()

31.0